In [1]:
# UNK 개수 새기

In [1]:
import pandas as pd
import re
import numpy as np
import pandas as pd
from multiprocessing import Pool
import multiprocessing


# get OOV rate

In [2]:
# [UNK]의 개수 / 문장의 길이 * 100
from typing import List

def getOOVRatePerSentence(sentence):
    # [CLS], [SEP] 제거
    sentence = sentence.split()
    try:
        sentence.remove('[CLS]')
        sentence.remove('[SEP]')
    except:
        pass
    OOV_rate = sentence.count('[UNK]') / len(sentence) * 100   
    
    return OOV_rate

In [3]:
# count of all OOV tokens (OC)
def getCountofallOOV(sentence):
    sentence = sentence.split()
    cnt = 0
    cnt += sentence.count('[UNK]')
    
    return cnt

In [4]:
# count of all sentences with OOV tokens, divided by the number of sentences * 100 (OR)
def getOOVsentencePerSentences(corpus): #corpus: 토큰화된 문장이 있는 칼럼
    cnt = 0
    for sentence in corpus:
        if '[UNK]' in sentence:
            cnt += 1
    OR = cnt / len(corpus)
    return cnt, len(corpus), OR

In [5]:
# [CLS, SEP] 제거
def removeCS(sentence):
    try:
        sentence.remove('[CLS]', '[SEP]')
    except:
        pass
    
    return sentence

In [6]:
# [UNK]수 /전체 토큰 수
def getOOVdividedbyAllTokens(corpus):
    corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))
    corpus['sentence'].apply(lambda x: removeCS(x))
    corpus['token_count'] = corpus['sentence'].apply(lambda x: len(x))
    OOV_count = corpus['OC'].sum()
    token_count = corpus['token_count'].sum()
    
    return OOV_count, token_count, OOV_count/token_count*100

In [8]:
# eojoel_composed
import pandas as pd
corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/eojeol_mecab_fixed_composed_grammatical_symbol_F_txt', 
                     sep='\t', names=['sentence'])
corpus.head()


,sentence
0,[CLS] 신 세계 ##수의 미궁 2에서 뜬 ! ##! ##아 ##앗 ##!! [SEP]
1,[CLS] 세계 ##수의 미궁 시리즈에 전통 ##으로 등장하는 대사 ##. 2편 #...
2,[CLS] 세계 ##수의 모험가 ##들이 탐험 ##하는 던전 ##인 수 ##해의 구...
3,[CLS] 그러나 분 ##배 ##할 수 있는 스킬 포인트 ##는 한정 ##되어 있기...
4,[CLS] 아 ##앗 ##!! ##이 발생하는 과정을 요약 ##하면 다음과 같다. ...


In [9]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

1905 4951697574 3.847165485232035e-05


In [20]:
OR = getOOVsentencePerSentences(corpus['sentence'])

In [21]:
OR

3.5657537187944736e-05

In [10]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

In [11]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [36]:
corpus.head()

,sentence,OOV_rate,OC
0,[CLS] 신 세계 ##수의 미궁 2에서 뜬 !! ##아 ##앗 ##!! [SEP],0.0,0
1,[CLS] 세계 ##수의 미궁 시리즈에 전통 ##으로 등장하는 대사. 2편 ##부터...,0.0,0
2,[CLS] 세계 ##수의 모험가 ##들이 탐험 ##하는 던전 ##인 수 ##해의 구...,0.0,0
3,[CLS] 그러나 분배 ##할 수 있는 스킬 포인트는 한정되어 있기 때문에 채집 스...,0.0,0
4,[CLS] 아 ##앗 ##!! ##이 발생하는 과정을 요약하면 다음과 같다. [SEP],0.0,0


In [12]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

0.00028828570606354324 
 3.689712566161582e-05 
 1905


In [13]:
# OOV 포함 문장 확인
print(corpus[corpus['sentence'].str.contains("[UNK]", regex=False, case=True)]['sentence'])

46772       [CLS] 바둑 ##의 모든 경우 ##의 가 ##짓 ##수는 36 ##1 ##! #...
223848      [CLS] 인터뷰에서 선수들의 대한 승부 ##조작 비난을 자제 ##해 달라고 [UN...
225328      [CLS] 미국 ##시간 10월 23일 ##, 뉴욕 에서 에 ##볼 ##라 확 ##...
229388        [CLS] 1라운드 및 한국 선수들의 활약 ##상을 보 ##려면 [UNK] [SEP]
229818      [CLS] [ ##* 사실 이와 비슷한 경주 전개 ##로 우리나라 ##가 금메달 #...
                                  ...                        
51567723    [CLS] 메인 예고 ##편 ##" w ##id ##th ##=" ##64 ##0 ...
51567724    [CLS] 론 ##칭 예고 ##편 ##" w ##id ##th ##=" ##64 #...
51568569    [CLS] [ ##* 논란 ##중인 사건의 [UNK] 중 윤 ##미래 ##라는 계정...
51604907    [CLS] 게이 ##미 ##피 ##케이 ##션 알바 ##를 게임 ##처럼 만들고 싶...
51614567    [CLS] [UNK] | ##[ ##F ##R ##] A ##vel ##ia H #...
Name: sentence, Length: 1841, dtype: object


In [14]:
# eojoel_pure
import pandas as pd

corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/eojeol_mecab_fixed_decomposed_pure_grammatical_symbol_F_txt', 
                 sep='\t', names=['sentence'])

In [15]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

3731 7310908250 5.1033330913433364e-05


In [21]:
OR = getOOVsentencePerSentences(corpus['sentence'])
print(OR)

(3564, 51630038, 6.902958312755841e-05)


In [16]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

In [17]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [18]:
corpus.head()

,sentence,OC,token_count,OOV_rate
0,[CLS] 신 세계 ##수의 미궁 2에서 뜬 ! ##! ##...,0,65,0.0
1,[CLS] 세계 ##수의 미궁 시리즈에 전통 ##으로...,0,141,0.0
2,[CLS] 세계 ##수의 모험가 ##들이 탐험 ##하ᄂ...,0,260,0.0
3,[CLS] 그러나 분배 ##할 수 있는 스킬 포인...,0,166,0.0
4,[CLS] 아아 ##ᆺ! ##! ##이 발생하는 과정을 ...,0,80,0.0


In [19]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

0.0006116152105434466 
 7.22641343010439e-05 
 3731


In [20]:
# OOV 포함 문장 확인
print(corpus[corpus['sentence'].str.contains("[UNK]", regex=False, case=True)]['sentence'])

39254                    [CLS] 주요 경유지 ##는 [UNK] [SEP]
46772       [CLS] 바둑 ##의 모든 경우의 가지 ##ᆺ수 ##느...
57625       [CLS] 스테이지 순서는 ##, [UNK] 1주차 ##이고...
177628      [CLS] 이집트 ##의 카이 ##로 ##-아 ##ᆯ레 ##ᆨ #...
213726             [CLS] [UNK] 갈 뻔 ##했는데 ##... [SEP]
                                  ...                        
51567723    [CLS] 메인 예고편 ##" width= ##" ##64 ##0 ##...
51567724    [CLS] 론 ##칭 예고편 ##" width= ##" ##64 ##...
51568569    [CLS] [ ##* 논란 ##중인 사건의 [UNK] 중 ...
51603429    [CLS] 사랑해 ##서, 사랑해 ##서, 사랑해 ##서...
51614567    [CLS] [UNK] | ##[ ##F ##R ##] A ##vel ##ia H #...
Name: sentence, Length: 3564, dtype: object


In [22]:
# orig_composed
corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/morpheme_mecab_orig_composed_grammatical_symbol_F_txt', 
                     sep='\t', names=['sentence'])

In [23]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

173 4407791098 3.924868401283749e-06


In [24]:
OR = getOOVsentencePerSentences(corpus['sentence'])
print(OR)

(153, 51630038, 2.963391194869932e-06)


In [25]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

In [26]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [27]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

1.9926507302732258e-05 
 3.3507625928921455e-06 
 173


In [28]:
corpus.head()

,sentence,OC,token_count,OOV_rate
0,[CLS] 신 세계수 의 미궁 2 에서 뜬 ! ! 아 ##앗 ! ! [SEP],0,43,0.0
1,[CLS] 세계수 의 미궁 시리즈 에 전통 으로 등장 하 는 대사 . 2 편 부터 ...,0,81,0.0
2,[CLS] 세계수 의 모험가 들 이 탐험 하 는 던전 인 수 ##해 의 구석 ##구...,0,151,0.0
3,[CLS] 그러나 분배 할 수 있 는 스킬 포인트 는 한정 되 어 있 기 때문 에 ...,0,96,0.0
4,[CLS] 아 ##앗 ! ! 이 발생 하 는 과정 을 요약 하 면 다음 과 같 다 ...,0,53,0.0


In [29]:
# OOV 포함 문장 확인
print(corpus[corpus['sentence'].str.contains("[UNK]", regex=False, case=True)]['sentence'])

2157462     [CLS] 위 에선 킥 킥 이 라고 하 지만 사람 에 따라 ' [UNK] ' ' 컄...
2166324     [CLS] 바리 에 이 션 으로 쌩 쏙 옙 , 싱 깍 옙 , [UNK] , 쌀 곤 ...
2186208     [CLS] 에 ##에 유메 오 ##치 ! ? 안 이나 야 ##츠 쟌 ! 안 이나 야...
2283509                     [CLS] 닭 이 언제 부터 [UNK] 울 었 지 [SEP]
2378438       [CLS] ㅎ 해 핼 해리 [UNK] ㅅ 시 ##실 시리 맂 ##리 ##즈 [SEP]
                                  ...                        
50690326    [CLS] 애초 에 횡단 보도 를 건 ##널 때 신호 ##등 도 안 보 고 차 들 ...
50812484    [CLS] 오늘 닐 기 ㅈ 댜 [UNK] 랴 ㅕ ##다 ##ㅗ ##ㄴ ##ㅇ ##ㅁ...
50867913           [CLS] 황 ##훈 {{{- 2 을 란 홍 더 [UNK] }}} [SEP]
50867938           [CLS] 황 ##훈 {{{- 2 을 란 홍 더 [UNK] }}} [SEP]
50918255            [CLS] 쿵 ##쾅 북 치 고 티 ##키 타카 헤드 [UNK] [SEP]
Name: sentence, Length: 153, dtype: object


In [10]:
# orig_pure
corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/morpheme_mecab_orig_decomposed_pure_grammatical_symbol_F_txt', 
                     sep='\t', names=['sentence'])

In [11]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

0 6931900436 0.0


In [12]:
OR = getOOVsentencePerSentences(corpus['sentence'])
print(OR)

(0, 51630038, 0.0)


In [13]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

In [14]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [15]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

0.0 
 0.0 
 0


In [16]:
corpus.head()

,sentence,OC,token_count,OOV_rate
0,[CLS] 신 세계수 의 미궁 2 에서 뜬 ! ! 아아 ...,0,59,0.0
1,[CLS] 세계수 의 미궁 시리즈 에 전통 으로 드...,0,137,0.0
2,[CLS] 세계수 의 모험가 들 이 탐험 하 는 ᄃ...,0,245,0.0
3,[CLS] 그러나 분배 할 수 있 는 스킬 포인ᄐ...,0,166,0.0
4,[CLS] 아아 ##ᆺ ! ! 이 발생 하 는 과정 을 ...,0,82,0.0


In [17]:
# OOV 포함 문장 확인
print(corpus[corpus['sentence'].str.contains("[UNK]", regex=False, case=True)]['sentence'])

Series([], Name: sentence, dtype: object)


In [18]:
# fixed_composed
corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/morpheme_mecab_fixed_composed_grammatical_symbol_F_txt', 
                     sep='\t', names=['sentence'])

In [19]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

176 4561042431 3.858766995978424e-06


In [20]:
OR = getOOVsentencePerSentences(corpus['sentence'])
print(OR)

(156, 51630038, 3.0214969045732643e-06)


In [21]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

In [22]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [23]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

1.9612320934193966e-05 
 3.4088683025954774e-06 
 176


In [24]:
corpus.head()

,sentence,OC,token_count,OOV_rate
0,[CLS] 신 세계수 의 미궁 2 에서 뜨 ᆫ ! ! 아 ##앗 ! ! [SEP],0,45,0.0
1,[CLS] 세계수 의 미궁 시리즈 에 전통 으로 등장 하 는 대사 . 2 편 부터 ...,0,86,0.0
2,[CLS] 세계수 의 모험가 들 이 탐험 하 는 던전 이 ᆫ 수 ##해 의 구석 #...,0,158,0.0
3,[CLS] 그러나 분배 하 ᆯ 수 있 는 스킬 포인트 는 한정 되 어 있 기 때문 ...,0,100,0.0
4,[CLS] 아 ##앗 ! ! 이 발생 하 는 과정 을 요약 하 면 다음 과 같 다 ...,0,53,0.0


In [25]:
# fixed_pure

corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/morpheme_mecab_fixed_decomposed_pure_grammatical_symbol_F_txt', 
                     sep='\t', names=['sentence'])

In [26]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

0 7083464382 0.0


In [27]:
OR = getOOVsentencePerSentences(corpus['sentence'])


In [28]:
print(OR)

(0, 51630038, 0.0)


In [29]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

In [30]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [31]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

0.0 
 0.0 
 0


In [32]:
corpus.head()

,sentence,OC,token_count,OOV_rate
0,[CLS] 신 세계수 의 미궁 2 에서 뜨 ᆫ ! ! 아아...,0,60,0.0
1,[CLS] 세계수 의 미궁 시리즈 에 전통 으로 드...,0,142,0.0
2,[CLS] 세계수 의 모험가 들 이 탐험 하 는 ᄃ...,0,250,0.0
3,[CLS] 그러나 분배 하 ᆯ 수 있 는 스킬 포인...,0,168,0.0
4,[CLS] 아아 ##ᆺ ! ! 이 발생 하 는 과정 을 ...,0,82,0.0


In [33]:
# OOV 포함 문장 확인
print(corpus[corpus['sentence'].str.contains("[UNK]", regex=False, case=True)]['sentence'])

Series([], Name: sentence, dtype: object)


In [7]:
#fixed_grammatical
corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/morpheme_mecab_fixed_decomposed_grammatical_grammatical_symbol_F_txt', 
                     sep='\t', names=['sentence'])

In [8]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

177 5207220770 3.399126094667194e-06


In [9]:
OR = getOOVsentencePerSentences(corpus['sentence'])

In [10]:
print(OR)

(157, 51630038, 3.0408654744743748e-06)


In [11]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

In [12]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [13]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

1.9580425523499977e-05 
 3.4282368724965883e-06 
 177


In [14]:
corpus.head()

,sentence,OC,token_count,OOV_rate
0,[CLS] 신 세계수 의 미궁 2 에서 뜨 ᆫ ! ! 아 ##앗 ! ! [SEP],0,48,0.0
1,[CLS] 세계수 의 미궁 시리즈 에 전통 으로 등장 하 는 대사 . 2...,0,100,0.0
2,[CLS] 세계수 의 모험가 들 이 탐험 하 는 던전 이 ᆫ 수 ##해 의...,0,179,0.0
3,[CLS] 그러나 분배 하 ᆯ 수 있 는 스킬 포인트 는 한정 되 어 있 ...,0,115,0.0
4,[CLS] 아 ##앗 ! ! 이 발생 하 는 과정 을 요약 하 면 다음...,0,62,0.0


In [15]:
# OOV 포함 문장 확인
print(corpus[corpus['sentence'].str.contains("[UNK]", regex=False, case=True)]['sentence'])

2157462     [CLS] 위 에서 ᆫ 킥 킥 이 라고 하 지만 사람 에 따르 아 ...
2166324     [CLS] 바리 에 이 션 으로 쌩 쏙 옙 , 싱 깍 옙 , [UNK] , 쌀...
2186208     [CLS] 에 ##에 유메 오 ##치 ! ? 안 이나 야 ##츠 쟌 ! 안 이...
2283509               [CLS] 닭 이 언제 부터 [UNK] 울 었 지 [SEP]
2378438     [CLS] ㅎ 하 아 핼 해리 [UNK] ㅅ 시 ##실 시리 맂 ##리 ##즈 [...
                                  ...                        
50690326    [CLS] 애초 에 횡단 보도 를 건너 ᆯ 때 신호 ##등 도 안 보 고 ...
50812484    [CLS] 오늘 닐 기 ㅈ 되 야 [UNK] 이 랴 ㅕ ##다 ##ㅗ ##ㄴ ...
50867913           [CLS] 황 ##훈 {{{- 2 을 란 홍 더 [UNK] }}} [SEP]
50867938           [CLS] 황 ##훈 {{{- 2 을 란 홍 더 [UNK] }}} [SEP]
50918255           [CLS] 쿵 ##쾅 북 치 고 티 ##키 타카 헤드 [UNK] [SEP]
Name: sentence, Length: 157, dtype: object


In [16]:
#fixed_lexical

In [17]:
corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/morpheme_mecab_fixed_decomposed_lexical_grammatical_symbol_F_txt', 
                     sep='\t', names=['sentence'])

In [18]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

31 6448150346 4.807580210847646e-07


In [19]:
OR = getOOVsentencePerSentences(corpus['sentence'])

In [20]:
print(OR)

(31, 51630038, 6.004256669344307e-07)


In [21]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

In [22]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [23]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

2.9228225692590487e-06 
 6.004256669344307e-07 
 31


In [24]:
corpus.head()

,sentence,OC,token_count,OOV_rate
0,[CLS] 신 세계수 의 미궁 2 에서 뜨 ᆫ ! ! 아아 ##...,0,57,0.0
1,[CLS] 세계수 의 미궁 시리즈 에 전통 으로 등장...,0,128,0.0
2,[CLS] 세계수 의 모험가 들 이 탐험 하 는 던저...,0,229,0.0
3,[CLS] 그러나 분배 하 ᆯ 수 있 는 스킬 포인트...,0,153,0.0
4,[CLS] 아아 ##ᆺ ! ! 이 발생 하 는 과정 을 요약...,0,73,0.0


In [25]:
# OOV 포함 문장 확인
print(corpus[corpus['sentence'].str.contains("[UNK]", regex=False, case=True)]['sentence'])

11597712    [CLS] 2018 년 리그 우승 축 승 ##회 에서 " 2 ...
29815786             [CLS] 나 의 사랑 정말 보이 [UNK] [SEP]
36286913    [CLS] 그리고 , SCP - 03 ##5 의 가면 을 쓰 면 HP ...
36792153    [CLS] * 그 가 그 학교 를 보 ᆫ ##즉 먹 음직 도 하 고...
44638460    [CLS] - 데이 : 히 ##ᆼ . .. 미안 하 ㅕ . 다음 ...
44692077    [CLS] 아까 다 하 ᆫ다 캐 었 [UNK] 서 러 이제 ᆫ 또 ...
44692078    [CLS] 이 문장 을 들 은 사람 들 은 듣 자마자 " ...
44692313       [CLS] 묵 [UNK] 서 러 하 아라 먹 으면서 하 아라 [SEP]
45698594    [CLS] # 000000 , # e 5 e 5 e 5 {{{- 2 안산 ᄃ...
48720006    [CLS] 주변 의 백발 에 희 ᆫ 눈동자 를 가지 고 ...
48823268    [CLS] 짓 - + - 민 = 짓 이민 / 짓 [UNK] [SEP]
48823270    [CLS] - 고 - 지 - [UNK] - 으 ##난 - 언 - 엇 수ᄃ...
48823271    [CLS] 잇 다 잇 고 잇 지 잇 이민 / 잇 [UNK] ...
48823275    [CLS] - 고 - 지 - [UNK] - 으 ##난 - 언 - 엇 수ᄃ...
48823276    [CLS] 엇 ##다 엇 고 엇 ##지 엇 이민 / 엇 [UNK...
48823278    [CLS] 없 다 없 고 없 지 없이 민 / 없 [UNK] ...
48823279

## T

In [26]:
# fixed_composed_T
corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/morpheme_mecab_fixed_composed_grammatical_symbol_T_txt', 
                     sep='\t', names=['sentence'])

In [27]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

177 5028068902 3.5202381560363116e-06


In [28]:
OR = getOOVsentencePerSentences(corpus['sentence'])
    
print(OR)

(157, 51630038, 3.0408654744743748e-06)


In [29]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

In [30]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [31]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

1.9822626368727764e-05 
 3.4282368724965883e-06 
 177


In [32]:
corpus.head()

,sentence,OC,token_count,OOV_rate
0,[CLS] 신 세계수 ⫸의 미궁 2 ⫸에서 뜨 ⭧ᆫ ! ! 아 ##앗 ! ! [SEP],0,48,0.0
1,[CLS] 세계수 ⫸의 미궁 시리즈 ⫸에 전통 ⫸으로 등장 하 ⭧는 대사 . 2 편...,0,96,0.0
2,[CLS] 세계수 ⫸의 모험가 들 ⫸이 탐험 하 ⭧는 던전 이 ⭧ᆫ 수 ##해 ⫸의...,0,177,0.0
3,[CLS] 그러나 분배 하 ⭧ᆯ 수 있 ⭧는 스킬 포인트 ⫸는 한정 되 ⭧어 있 ⭧...,0,112,0.0
4,[CLS] 아 ##앗 ! ! ⫸이 발생 하 ⭧는 과정 ⫸을 요약 하 ⭧면 다음 ⫸과...,0,59,0.0


In [33]:
# OOV 포함 문장 확인
print(corpus[corpus['sentence'].str.contains("[UNK]", regex=False, case=True)]['sentence'])

2157462     [CLS] 위 ⫸에서 ⫸ᆫ 킥 킥 이 ⭧라고 하 ⭧지만 사람 ⫸에 따르 ⭧아 ' [...
2166324     [CLS] 바리 ⫸에 이 션 ⫸으로 쌩 쏙 옙 , 싱 깍 옙 , [UNK] , 쌀 ...
2186208     [CLS] 에 ##에 유메 오 ##치 ! ? 안 ⫸이나 야 ##츠 쟌 ! 안 ⫸이나...
2283509                 [CLS] 닭 ⫸이 언제 ⫸부터 [UNK] 울 ⭧었 ⭧지 [SEP]
2378438     [CLS] ㅎ 하 ⭧아 핼 해리 [UNK] ㅅ 시 ##실 시리 맂 ##리 ##즈 [...
                                  ...                        
50690326    [CLS] 애초 ⫸에 횡단 보도 ⫸를 건너 ⭧ᆯ 때 신호 ##등 ⫸도 안 보 ⭧고 ...
50812484    [CLS] 오늘 닐 ⭧기 ㅈ 되 ⭧야 [UNK] 이 ⭧ ##랴 ㅕ ##다 ##ㅗ #...
50867913           [CLS] 황 ##훈 {{{- 2 을 란 홍 더 [UNK] }}} [SEP]
50867938           [CLS] 황 ##훈 {{{- 2 을 란 홍 더 [UNK] }}} [SEP]
50918255           [CLS] 쿵 ##쾅 북 치 ⭧고 티 ##키 타카 헤드 [UNK] [SEP]
Name: sentence, Length: 157, dtype: object


In [10]:
# fixed_pure_T
corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/morpheme_mecab_fixed_decomposed_pure_grammatical_symbol_T_txt', 
                     sep='\t', names=['sentence'])

In [11]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

0 7547752175 0.0


In [ ]:
OR = getOOVsentencePerSentences(corpus['sentence'])
    
print(OR)

In [12]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

KeyboardInterrupt: 

In [ ]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [ ]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

In [ ]:
corpus.head()

In [ ]:
# OOV 포함 문장 확인
print(corpus[corpus['sentence'].str.contains("[UNK]", regex=False, case=True)]['sentence'])

In [ ]:
# fixed_grammatical_T

In [14]:
corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/morpheme_mecab_fixed_decomposed_grammatical_grammatical_symbol_T_txt', 
                     sep='\t', names=['sentence'])

In [15]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

177 5671062940 3.121108015775258e-06


In [ ]:
OR = getOOVsentencePerSentences(corpus['sentence'])
    
print(OR)

In [16]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

In [17]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [18]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

1.9524011690390983e-05 
 3.4282368724965883e-06 
 177


In [19]:
corpus.head()

,sentence,OC,token_count,OOV_rate
0,[CLS] 신 세계수 ⫸의 미궁 2 ⫸에서 뜨 ⭧ᆫ ! ! 아 ##앗 ! ! ...,0,51,0.0
1,[CLS] 세계수 ⫸의 미궁 시리즈 ⫸에 전통 ⫸으로 등장 하 ⭧는 대사...,0,110,0.0
2,[CLS] 세계수 ⫸의 모험가 들 ⫸이 탐험 하 ⭧는 던전 이 ⭧ᆫ 수 ##...,0,198,0.0
3,[CLS] 그러나 분배 하 ⭧ᆯ 수 있 ⭧는 스킬 포인트 ⫸는 한정 되 ⭧ᄋ...,0,127,0.0
4,[CLS] 아 ##앗 ! ! ⫸이 발생 하 ⭧는 과정 ⫸을 요약 하 ⭧며...,0,68,0.0


In [20]:
# OOV 포함 문장 확인
print(corpus[corpus['sentence'].str.contains("[UNK]", regex=False, case=True)]['sentence'])

2157462     [CLS] 위 ⫸에서 ⫸ᆫ 킥 킥 이 ⭧라고 하 ⭧지만 사람 ⫸에 따...
2166324     [CLS] 바리 ⫸에 이 션 ⫸으로 쌩 쏙 옙 , 싱 깍 옙 , [UNK] ,...
2186208     [CLS] 에 ##에 유메 오 ##치 ! ? 안 ⫸이나 야 ##츠 쟌 ! 안 ⫸...
2283509           [CLS] 닭 ⫸이 언제 ⫸부터 [UNK] 울 ⭧었 ⭧지 [SEP]
2378438     [CLS] ㅎ 하 ⭧아 핼 해리 [UNK] ㅅ 시 ##실 시리 맂 ##리 ##즈 ...
                                  ...                        
50690326    [CLS] 애초 ⫸에 횡단 보도 ⫸를 건너 ⭧ᆯ 때 신호 ##등 ⫸도 안 보...
50812484    [CLS] 오늘 닐 ⭧기 ㅈ 되 ⭧야 [UNK] 이 ⭧ᄅ ##ᅣ ㅕ ##다 ##...
50867913           [CLS] 황 ##훈 {{{- 2 을 란 홍 더 [UNK] }}} [SEP]
50867938           [CLS] 황 ##훈 {{{- 2 을 란 홍 더 [UNK] }}} [SEP]
50918255          [CLS] 쿵 ##쾅 북 치 ⭧고 티 ##키 타카 헤드 [UNK] [SEP]
Name: sentence, Length: 157, dtype: object


In [7]:
# fixed_lexical_T
corpus = pd.read_csv('/run/user/1000/gvfs/afp-volume:host=L8221.local,user=bongseok,volume=공유폴더/양봉석/bpe_tokenized/32k/morpheme_mecab_fixed_decomposed_lexical_grammatical_symbol_T_txt', 
                     sep='\t', names=['sentence'])

In [8]:
oov_count, token_count, rate = getOOVdividedbyAllTokens(corpus)
print(oov_count, token_count, rate)

5 6913795991 7.231917179084724e-08


In [ ]:
OR = getOOVsentencePerSentences(corpus['sentence'])
    
print(OR)

In [9]:
corpus['OOV_rate'] = corpus['sentence'].apply(lambda x: getOOVRatePerSentence(x))

In [10]:
corpus['OC'] = corpus['sentence'].apply(lambda x: getCountofallOOV(x))

In [11]:
print(corpus['OOV_rate'].mean(), '\n', corpus['OC'].mean(), '\n', corpus['OC'].sum())

5.308983338222993e-07 
 9.684284950555333e-08 
 5


In [12]:
corpus.head()

,sentence,OC,token_count,OOV_rate
0,[CLS] 신 세계수 ⫸의 미궁 2 ⫸에서 뜨 ⭧ᆫ ! ! 아아...,0,60,0.0
1,[CLS] 세계수 ⫸의 미궁 시리즈 ⫸에 전통 ⫸으로 등...,0,138,0.0
2,[CLS] 세계수 ⫸의 모험가 들 ⫸이 탐험 하 ⭧는 더...,0,248,0.0
3,[CLS] 그러나 분배 하 ⭧ᆯ 수 있 ⭧는 스킬 포인...,0,165,0.0
4,[CLS] 아아 ##ᆺ ! ! ⫸이 발생 하 ⭧는 과정 ⫸을 요...,0,79,0.0


In [13]:
# OOV 포함 문장 확인
print(corpus[corpus['sentence'].str.contains("[UNK]", regex=False, case=True)]['sentence'])

29815786            [CLS] 나 ⫸의 사랑 정말 보이 [UNK] [SEP]
36286913    [CLS] 그리고 , SCP - 03 ##5 ⫸의 가면 ⫸을 쓰 ⭧면 ...
36792153    [CLS] * 그 ⫸가 그 학교 ⫸를 보 ⭧ᆫ ##즉 먹 ⭧음직 ⫸도...
44638460    [CLS] - 데이 : 히 ##ᆼ . .. 미안 하 ⭧ㅕ . 다음...
48720006    [CLS] 주변 ⫸의 백발 ⫸에 희 ⭧ᆫ 눈동자 ⫸를 가ᄌ...
Name: sentence, dtype: object
